In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import math 
import time
import numpy as np
from collections import Counter
from tqdm import tqdm
import array as arr
import copy
from multiprocessing import Pool
from itertools import repeat
import tqdm
import AmortizedRandomWalk

filepathTXT = "../edgelists/BlogCatalog-edgelist.txt"
filepathCSV = "../edgelists/BlogCatalog-edgelist.csv"
embeddingsrecursive = "../embeddings/BlogCatalog-edgelist.txt.embeddings-recursive"
embeddingsiterative = "../embeddings/BlogCatalog-edgelist.txt.embeddings-iterative"

In [ ]:
def parseEdgeList2(graph_file, direction="undirected"):
    # Create Graph
    G = nx.Graph()
    # Create head
    colNames=["Start", "End"]
    edgeData = pd.read_csv(filepathCSV, names=colNames)

    #Add nodes
    nodes = []
    #loop throug data records
    for i in range (0, edgeData.shape[0]):
        #append every node
        nodes.append(edgeData.iloc[i,0])
        nodes.append(edgeData.iloc[i,1])
    #creating a set of nodes    
    nodes = set(nodes)
    #sorting the nodes in increasing order
    uniqueNodes = (list(nodes))
    uniqueNodes.sort()
    #adding the nodes to the graph
    G.add_nodes_from(uniqueNodes)

    # Add edges
    #loop from 0 to amount of records
    edgeCount = 0
    for i in range (0, edgeData.shape[0]):
        edgeCount += 1
        #add the edge to the graph
        G.add_edge(edgeData.iloc[i,0], edgeData.iloc[i,1])
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges(), " loaded from ", graph_file)

    if(direction == "undirected"):
        return G.to_undirected()
    else:
        return G

In [ ]:
def getAdjNPList(graph):
    adjdict = {}
    for vertex in graph:
        adjdict[vertex] = np.array([n for n in G.neighbors(vertex)]) 
        np.random.shuffle(adjdict[vertex])
    return adjdict


In [ ]:
femb_recursive = open(embeddingsrecursive, 'w')
femb_iterative = open(embeddingsiterative, 'w')

## Toy Graph

In [ ]:
def toyGraph():
    G = nx.Graph()
    G.add_nodes_from([1,2,3,4,5,6,7,8,9,10,11])
    G.add_edge(2, 1);G.add_edge(3, 1);G.add_edge(3, 2);G.add_edge(4, 1);
    G.add_edge(4, 2);G.add_edge(4, 3);G.add_edge(5, 1);G.add_edge(6, 1);
    G.add_edge(7, 1);G.add_edge(7, 5);G.add_edge(7, 6);G.add_edge(8, 1);
    G.add_edge(8, 2);G.add_edge(8, 3);G.add_edge(8, 4);G.add_edge(9, 1); 
    G.add_edge(9, 3);G.add_edge(10, 3);G.add_edge(11, 1);G.add_edge(11, 5);
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges())
    # Draw graph
    # nx.draw(G, with_labels = True)
    # plt.show()
    return G

### NP implementation

In [ ]:
def chooseNodes(list_nodes, sample_size):
    return random.sample(population=list_nodes, k=sample_size) # 2.22 s ± 179 ms per loop  (labesl str)

def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] +  budget
        
def updateContextPairs(window, window_count, context_pairs):
    lastNode = window[window_count]
    labelOfLastNode, budgetOfLastNode = lastNode
    index_count = 0
    window_except_last = window[:window_count]
    window_reversed = window_except_last[::-1]
    for node in window_reversed:
        node_label = node[0]
        if index_count == ORIGINAL_WINDOW_SIZE:
            break
        context_pair1 = (labelOfLastNode, node_label)
        context_pairs.append(context_pair1)
        
        context_pair2 = (node_label, labelOfLastNode)
        context_pairs.append(context_pair2)
        
        index_count = index_count + 1
    return context_pairs

def addNewNodeToWindow(tempwindow, temp_window_count, window_size, vertex, budget):
    newWindowElement = np.array([[vertex,budget]])
    if temp_window_count+1 == window_size:
        tempwindow = tempwindow[1:]
        tempwindow[temp_window_count] = newWindowElement
        return tempwindow, temp_window_count

    tempwindow[temp_window_count+1] = newWindowElement
    temp_window_count+=1
    return tempwindow, temp_window_count

def BFSRandomWalkWindow(startvertex):
    context_pairs = []
    
    window_count = 0
    WINDOW = np.zeros(shape=(WINDOW_SIZE+20,2),dtype=int)
    firstWindowElement = np.array([[startvertex, BUDGET]])
    WINDOW[window_count] = firstWindowElement

    queue_len = 0
    queue_pop_idx = 0
    queue_add_idx = 0
    queue = np.zeros(shape=(BUDGET,5),dtype=object)
    firstQueueElement = np.array([[startvertex, BUDGET, 1, window_count, WINDOW]])
    queue[queue_len] = firstQueueElement
    queue_len+=1
    
    queue_buffer_size = (queue.size/5)-1
    
    while queue_len > 0:
        vertex, budget, current_walk_lenght, window_count, window = queue[queue_pop_idx] 
        queue_len -=1
        if queue_len > 0:
            if queue_pop_idx == queue_buffer_size:
                queue_pop_idx = 0
            else:
                queue_pop_idx += 1
        vertex_neighbors = adjdict[vertex]
        num_neighbors = vertex_neighbors.size
        m = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (m * num_neighbors)
        if remainder > 0:
            np.random.shuffle(vertex_neighbors) 
        else:
            remainder = num_neighbors 
        current_walk_lenght += 1
        for neighbor in vertex_neighbors[:remainder]:
            budget_for_this_node = m 
            temp_window = np.copy(window)
            temp_window_count = window_count
            if remainder != num_neighbors:
                budget_for_this_node = budget_for_this_node + 1
            temp_window, temp_window_count = addNewNodeToWindow(temp_window, temp_window_count, WINDOW_SIZE, neighbor, budget_for_this_node) 
            context_pairs = updateContextPairs(temp_window, temp_window_count, context_pairs) 
            if current_walk_lenght < WALK_LENGHT:
                newQueueElement = np.array([[neighbor, budget_for_this_node, current_walk_lenght, temp_window_count, temp_window]])
                queue[queue_add_idx] = newQueueElement
                if queue_add_idx == queue_buffer_size:
                    queue_add_idx = 0
                else:
                    queue_add_idx += 1
                queue_len+=1
    return context_pairs

### Parametrs

In [ ]:
# Set toy parameters and graph
# WALK_LENGHT = 3
# BUDGET = 1
# ORIGINAL_WINDOW_SIZE = 1
# WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1

G = toyGraph()
adjdict = getAdjNPList(G)

In [ ]:
# Set the actual parameters and graph
# WALK_LENGHT = 40
# BUDGET = 80
# ORIGINAL_WINDOW_SIZE = 10
# WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1

G = parseEdgeList2(filepathCSV) # Integer labels
adjdict = getAdjNPList(G)

## NP Runner

In [ ]:
def Runner():   
    all_context_pairs = []
    for startvertex in adjdict.keys(): 
        context_pairs = BFSRandomWalkWindow(startvertex)
        all_context_pairs.extend(context_pairs)
    return all_context_pairs

## Run

In [ ]:
contextPairs = Runner()

In [ ]:
# Count the total count sum
countSum = 0
for key, value in contextPairs.items():
    countSum += value
print("Total value sums up to: ", countSum)

In [ ]:
# Count the total count sum
print(len(contextPairs))

In [ ]:
contextPairs

### Writing to file

In [ ]:
print("Writing context pairs to file...")    
#Writing to file    
for (key, value) in contextPairs.items():
    femb_iterative.write(str(key) + " " + str(value) + "\n" )
femb_iterative.close()

### Multiprocessing

In [ ]:
# Test
import worker
if __name__ ==  '__main__': 
    num_processors = 3
    a = 5
    p = Pool(processes = num_processors)
    data = [(1,2),(3,4),(5,6),(7,8)]
    out1 = p.map(worker.worker,([i for i in data]))
    print(out1)

In [ ]:

if __name__ ==  '__main__': 
    start = time.time()
    num_processors = 4
    p = Pool(processes = num_processors)
    nodes = [i for i in adjdict.keys()]
    contextPairs = p.starmap(AmortizedRandomWalk.BFSRandomWalkWindow, zip(nodes, repeat(adjdict)))
    end = time.time()
    result = end - start
    print("The execution time ->", result)

In [ ]:
len(contextPairs)

In [ ]:
total_count = 0
for cp_set in contextPairs:
    total_count += len(cp_set)
print(total_count)
    